In [2]:
!pip install fastapi uvicorn pyngrok tensorflow opencv-python
!pip install python-multipart

In [3]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import tensorflow as tf
import numpy as np
import cv2
from io import BytesIO
from PIL import Image
from pyngrok import ngrok
import uvicorn
import threading


In [4]:
# CELL 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# CELL 2: Load Model
model = tf.keras.models.load_model('/content/drive/MyDrive/mymobilenet_final_model_for_tumour.keras')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ngrok config add-authtoken Ngrok_AuthToken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
!pkill -9 ngrok

In [9]:
# CELL 3: Define FastAPI app
app = FastAPI()

# CELL 4: Class Labels
class_labels = ['G', 'R', 'U']

# CELL 5: Define Prediction Endpoint
@app.post("/predict")
async def predict_tumor(file: UploadFile = File(...)):
    try:
        # Read image
        image_bytes = await file.read()
        image = Image.open(BytesIO(image_bytes)).convert("RGB")
        image = image.resize((224, 224))

        # Convert image to array
        img_array = np.array(image) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Make prediction
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions)

        return JSONResponse(content={"Prediction": class_labels[predicted_class]})
    except Exception as e:
        return JSONResponse(content={"error": str(e)})

# CELL 6: Start ngrok to get a public URL
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

# CELL 7: Run the API server in a separate thread
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)
threading.Thread(target=run_api, daemon=True).start()


Public URL: https://e689-34-105-50-12.ngrok-free.app
